<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1


    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-18 15:01:01
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  6.17 L
Current:  1.43 C
-------------------
Today PnL: 26.81 K (0.19%)
Current PnL: -16.80 L (-11.3%)
CY Booked + Current PnL: -5.36 L (-3.6%)
-------------------
Total profit:  2.08 L
Total loss:  -18.88 L
-------------------
Total Booked + Current PnL: 21.62 L (17.77%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.01%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.24 C
Est FTT PnL: 81.33 L (56.95%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.91%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.57,196410.0,31740.0,11588.0,1.23,19.27,5.90,26.31,89.0,2.74,1.44,28.80,X40N,NTT,AC
76,TTKPRESTIG,770.00,104.48,57.0,M-SC,3.97,88065.0,-12712.0,12805.0,-0.66,-12.61,14.54,0.09,245.0,-0.99,0.64,15.70,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.86,61.0,H-MC,2.51,213600.0,21858.0,15892.0,0.86,11.40,7.44,19.69,99.0,1.38,1.56,18.97,XY25,NTT,AC
49,NATIONALUM,247.44,-42.83,65.0,H-MC,6.84,113340.0,12836.0,17806.0,0.28,12.77,15.71,30.49,79.0,0.72,0.83,49.32,MH,ATH,METALS
56,RELIANCE,1533.00,-11.79,59.0,H-LC,5.18,220553.0,10187.0,18593.0,0.00,4.84,8.43,13.68,37.0,0.55,1.61,22.17,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,IEX,219.00,-29.46,56.0,H-SC,17.06,201664.0,1710.0,95528.0,0.11,0.86,47.37,48.63,115.0,0.02,1.48,12.51,XR,NTT,MISC
34,ICICIPRULI,790.00,-20.42,47.0,H-MC,2.22,136854.0,1066.0,41686.0,0.91,0.79,30.46,31.48,107.0,0.03,1.00,13.00,X40,ATH,INSURANCE
29,HAVELLS,2069.16,0.06,58.0,H-MC,1.87,248180.0,449.0,74603.0,-0.16,0.18,30.06,30.30,92.0,0.01,1.82,13.50,X40,ATH,ELECTRICAL
51,PGHH,17973.08,-30.68,53.0,H-MC,4.20,200625.0,-195.0,68975.0,-0.07,-0.10,34.38,34.25,80.0,-0.00,1.47,5.10,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VBL,671.64,-16.13,42.0,H-LC,6.34,301870.0,-13972.0,126634.0,0.45,-4.42,41.95,35.67,5.0,-0.11,2.21,8.82,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-13.71,44.0,H-LC,7.79,217923.0,-33845.0,87060.0,-0.69,-13.44,39.95,21.13,27.0,-0.39,1.59,17.22,X40,ATH,PAINTS
79,UNITDSPR,1644.00,-12.94,52.0,H-LC,7.80,228330.0,-5814.0,54434.0,-0.72,-2.48,23.84,20.77,86.0,-0.11,1.67,4.26,X40N,NTT,BREWERIES
41,ITC,452.00,-37.50,52.0,H-LC,2.52,201297.0,1159.0,19727.0,0.57,0.58,9.80,10.44,4.0,0.06,1.47,6.17,X40,NTT,FMCG
43,JIOFIN,387.00,0.39,53.0,H-LC,13.23,272792.0,5427.0,60042.0,0.43,2.03,22.01,24.48,48.0,0.09,2.00,58.12,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,LTIM,7230.2,0.00,70.0,H-LC,1.27,248130.0,1979.0,77218.0,1.68,0.80,31.12,32.18,16.0,0.03,1.82,37.14,X200,ATH,IT
41,ITC,452.0,-37.50,52.0,H-LC,2.52,201297.0,1159.0,19727.0,0.57,0.58,9.80,10.44,4.0,0.06,1.47,6.17,X40,NTT,FMCG
50,NESTLEIND,1377.0,-8.35,57.0,H-LC,3.85,282087.0,16661.0,40141.0,0.10,6.28,14.23,21.40,11.0,0.42,2.06,13.34,XY25,NTT,FMCG
56,RELIANCE,1533.0,-11.79,59.0,H-LC,5.18,220553.0,10187.0,18593.0,0.00,4.84,8.43,13.68,37.0,0.55,1.61,22.17,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-19.09,56.0,H-LC,5.81,216159.0,11659.0,29765.0,1.17,5.70,13.77,20.25,10.0,0.39,1.58,14.93,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6888.89,61.0,L-SC,5.24,76693.0,-17117.0,93734.0,3.51,-18.25,122.22,81.67,269.0,-0.18,0.56,48.52,XR,NTT,CERAMICS
63,SHALBY,327.00,1097.77,68.0,M-SC,3.64,162855.0,-18288.0,63758.0,3.14,-10.10,39.15,25.10,235.0,-0.29,1.19,30.51,XY24,NTT,HEALTHCARE
60,SAMMAANCAP,326.00,-172.73,65.0,M-SC,2.67,83502.0,-18708.0,112101.0,3.04,-18.30,134.25,91.37,208.0,-0.17,0.61,35.55,XY25,NTT,FINANCE
15,BSOFT,836.99,-11.70,58.0,H-SC,15.01,99170.0,-41483.0,114264.0,2.07,-29.49,115.22,51.74,131.0,-0.36,0.73,13.09,XR,ATH,IT
46,LAOPALA,464.00,116.69,53.0,H-SC,2.37,99028.0,-32077.0,83322.0,1.76,-24.47,84.14,39.09,142.0,-0.38,0.72,27.54,AR,NTT,CERAMICS


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RAJOOENG,143.33,-37.21,35.0,H-SC,11.65,95210.0,-7290.0,48119.0,-2.85,-7.11,50.54,39.83,114.0,-0.15,0.70,0.00,AR,ATH,MISC
26,GREENPANEL,537.00,229.46,60.0,M-SC,2.50,147108.0,-35970.0,114950.0,-2.74,-19.65,78.14,43.14,230.0,-0.31,1.08,38.38,XY24,NTT,MISC
57,REPCOHOME,880.00,-59.76,52.0,H-SC,6.43,180787.0,-54956.0,244261.0,-1.64,-23.31,135.11,80.30,134.0,-0.22,1.32,22.38,XY24,NTT,FINANCE
20,COALINDIA,497.97,8.96,60.0,L-LC,2.37,145595.0,-4866.0,38655.0,-1.56,-3.23,26.55,22.46,167.0,-0.13,1.07,14.26,XY25,ATH,MINING
72,TATAMOTORS,1065.00,-18.21,57.0,M-LC,2.04,257803.0,-18720.0,128798.0,-1.24,-6.77,49.96,39.81,54.0,-0.15,1.89,23.55,XY24,NTT,AUTO


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,104.48,57.0,M-SC,3.97,88065.0,-12712.0,12805.0,-0.66,-12.61,14.54,0.09,245.0,-0.99,0.64,15.70,OX40N,NTT,DURABLES
77,UJJIVANSFB,60.0,94.87,54.0,H-SC,9.52,123039.0,-19440.0,38966.0,-0.39,-13.64,31.67,13.70,163.0,-0.50,0.90,45.22,OX40N,NTT,BANKS
44,KANSAINER,340.0,-67.06,64.0,H-SC,1.39,227655.0,-42012.0,78336.0,1.39,-15.58,34.41,13.47,138.0,-0.54,1.67,15.14,XY24,NTT,PAINTS
18,CERA,9475.0,-19.81,47.0,H-SC,2.10,146062.0,-29841.0,71863.0,-0.37,-16.96,49.20,23.89,149.0,-0.42,1.07,26.14,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-19.51,54.0,H-MC,7.65,108918.0,-22917.0,65144.0,0.87,-17.38,59.81,32.03,98.0,-0.35,0.80,22.00,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4476.75,-25.15,59.0,H-LC,12.58,298243.0,-47713.0,122578.0,0.86,-13.79,41.10,21.64,1.0,-0.39,2.18,5.65,X40,ATH,IT
39,INFY,2275.00,-15.65,58.0,H-LC,8.41,328425.0,15537.0,156166.0,1.28,4.97,47.55,54.87,3.0,0.10,2.40,11.95,X40,BTT,IT
41,ITC,452.00,-37.50,52.0,H-LC,2.52,201297.0,1159.0,19727.0,0.57,0.58,9.80,10.44,4.0,0.06,1.47,6.17,X40,NTT,FMCG
82,VBL,671.64,-16.13,42.0,H-LC,6.34,301870.0,-13972.0,126634.0,0.45,-4.42,41.95,35.67,5.0,-0.11,2.21,8.82,X40N,ATH,FMCG
1,ABB,7934.00,-37.53,68.0,H-LC,10.33,260966.0,-653.0,119862.0,0.95,-0.25,45.93,45.57,7.0,-0.01,1.91,11.04,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,SAIL,228.00,45.15,65.0,M-MC,6.97,228664.0,3702.0,163952.0,0.03,1.65,71.70,74.53,192.0,0.02,1.67,34.43,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,65.0,H-MC,3.57,196410.0,31740.0,11588.0,1.23,19.27,5.90,26.31,89.0,2.74,1.44,28.80,X40N,NTT,AC
83,VOLTAS,1530.00,1.86,61.0,H-MC,2.51,213600.0,21858.0,15892.0,0.86,11.40,7.44,19.69,99.0,1.38,1.56,18.97,XY25,NTT,AC
17,CAMS,5211.76,0.38,67.0,H-SC,3.81,114100.0,12096.0,31834.0,1.53,11.86,27.90,43.06,122.0,0.38,0.84,31.00,X40N,ATH,MISC
29,HAVELLS,2069.16,0.06,58.0,H-MC,1.87,248180.0,449.0,74603.0,-0.16,0.18,30.06,30.30,92.0,0.01,1.82,13.50,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,NATIONALUM,247.44,-42.83,65.0,H-MC,6.84,113340.0,12836.0,17806.0,0.28,12.77,15.71,30.49,79.0,0.72,0.83,49.32,MH,ATH,METALS
84,WHIRLPOOL,2270.00,-40.92,50.0,M-SC,0.47,99908.0,8410.0,70345.0,-0.71,9.19,70.41,86.07,223.0,0.12,0.73,45.08,XR,NTT,DURABLES
36,INDIAMART,4850.92,-52.21,40.0,H-SC,11.64,133560.0,10224.0,123543.0,-1.21,8.29,92.50,108.45,119.0,0.08,0.98,32.50,AR,ATH,MISC
85,WIPRO,420.00,-9.26,60.0,M-LC,5.63,159278.0,8333.0,101126.0,1.08,5.52,63.49,72.51,53.0,0.08,1.17,11.88,XR,NTT,IT
11,BANKINDIA,190.00,-29.19,63.0,H-MC,6.11,183084.0,3276.0,109136.0,-0.50,1.82,59.61,62.52,88.0,0.03,1.34,36.53,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,RAJESHEXPO,518.00,1891.16,67.0,L-SC,3.06,54325.0,-82852.0,82943.0,0.08,-60.40,152.68,0.07,267.0,-1.00,0.40,35.11,OX40N,NTT,JEWELLERY
6,ASIANTILES,137.00,6888.89,61.0,L-SC,5.24,76693.0,-17117.0,93734.0,3.51,-18.25,122.22,81.67,269.0,-0.18,0.56,48.52,XR,NTT,CERAMICS
60,SAMMAANCAP,326.00,-172.73,65.0,M-SC,2.67,83502.0,-18708.0,112101.0,3.04,-18.30,134.25,91.37,208.0,-0.17,0.61,35.55,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-43.66,52.0,L-SC,42.31,84864.0,-28685.0,68740.0,-0.29,-25.26,81.00,35.27,268.0,-0.42,0.62,107.71,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-4.00,49.0,H-SC,12.75,89376.0,-11587.0,106116.0,-0.73,-11.48,118.73,93.63,148.0,-0.11,0.65,33.12,SR,ATH,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.0,3921.62,64.0,M-SC,5.81,125340.0,-22028.0,73850.0,1.42,-14.95,58.92,35.16,236.0,-0.30,0.92,28.02,XY24,NTT,AUTO
60,SAMMAANCAP,326.0,-172.73,65.0,M-SC,2.67,83502.0,-18708.0,112101.0,3.04,-18.30,134.25,91.37,208.0,-0.17,0.61,35.55,XY25,NTT,FINANCE
63,SHALBY,327.0,1097.77,68.0,M-SC,3.64,162855.0,-18288.0,63758.0,3.14,-10.10,39.15,25.10,235.0,-0.29,1.19,30.51,XY24,NTT,HEALTHCARE
21,COFFEEDAY,80.0,-43.66,52.0,L-SC,42.31,84864.0,-28685.0,68740.0,-0.29,-25.26,81.00,35.27,268.0,-0.42,0.62,107.71,XR,NTT,HOTELS
80,VAIBHAVGBL,521.0,65.84,71.0,H-SC,7.26,143679.0,-39096.0,150159.0,0.62,-21.39,104.51,60.77,125.0,-0.26,1.05,28.69,XR,NTT,JEWELLERY


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.11
1,25,43.91
2,50,74.48


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.56
LC    32.16
MC    23.30
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.28
X40      15.43
X40N     11.99
XR       11.79
AR        8.76
XY25      8.26
OX40N     8.22
X200      1.82
MH        1.72
X5K       1.45
SR        1.30
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.31
H-LC    25.44
H-MC    20.33
M-SC    13.67
M-LC     5.65
M-MC     2.66
L-SC     1.58
L-LC     1.07
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.71,-12.35,73.61
FMCG,11.91,-3.99,40.38
FINANCE,9.74,-15.60,60.87
MISC,7.49,-12.63,75.27
BANKS,6.31,-15.91,76.89
PAINTS,5.86,-15.25,32.42
ELECTRICAL,5.78,-4.47,62.04
AC,3.72,4.67,11.29
INSURANCE,3.44,-6.76,43.38


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2844962.0,22
XR,1300581.0,14
AR,1300507.0,9
X40,739769.0,10
X40N,568633.0,9
OX40N,523086.0,10
XY25,334602.0,6
SR,243610.0,2
X5K,119725.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3408810.0,29
M-SC,1378955.0,17
H-LC,1215332.0,14
H-MC,1092423.0,15
M-LC,379696.0,4
M-MC,313379.0,2
L-SC,245417.0,3
L-MC,60698.0,1
L-LC,38655.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1027680.0      6
M-SC       XY24       795169.0      7
H-SC       AR         787655.0      5
           XR         752091.0      7
H-MC       XY24       489847.0      4
H-LC       X40        456287.0      5
           AR         352218.0      2
M-MC       XY24       313379.0      2
H-SC       X40N       285352.0      4
           OX40N      249556.0      4
           SR         243610.0      2
H-MC       X40        212930.0      4
H-LC       X40N       210833.0      3
L-SC       XR         162474.0      2
M-SC       AR         160634.0      2
M-LC       XY24       158845.0      2
M-SC       OX40N      125443.0      4
H-MC       XY25       125112.0      2
M-LC       X5K        119725.0      1
M-SC       XR         115056.0      2
           XY25       112101.0      1
H-MC       XR         109136.0      1
M-LC       XR         101126.0      1
L-SC       OX40N       82943.0      1
H-LC       X200        77218.0      1
H-MC       X40N        72448.0      2
M-SC       X40         70552.0      1
H-MC       OX40N       65144.0      1
H-SC       MH          62866.0      1
L-MC       XR          60698.0      1
H-LC       XY24        60042.0      1
           XY25        58734.0      2
L-LC       XY25        38655.0      1
H-MC       MH          17806.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 54.0 seconds
